# 重回帰分析の実装 (NumPy)

このノートブックでは、NumPyのみを使用して重回帰分析をスクラッチから実装します。
最小二乗法を用いて回帰係数を推定します。

## 1. ライブラリのインポート

In [1]:
import numpy as np

## 2. データセットの準備

簡単なサンプルデータを作成してテストします。

*   `X_sample`: 2つの説明変数を持つ特徴量データ
*   `y_sample`: 目的変数

In [2]:
# サンプルデータ
# X は (サンプル数, 特徴量数) の形状
X_train_sample = np.array([
    [1, 1],
    [1, 2],
    [2, 2],
    [2, 3],
    [3, 2],
    [3, 4],
    [4, 4],
    [4, 5]
])
# y は (サンプル数,) の形状
# y = 2*X1 + 1*X2 + random noise
y_train_sample = 2 * X_train_sample[:, 0] + 1 * X_train_sample[:, 1] + np.random.normal(0, 0.2, X_train_sample.shape[0])

print("サンプル特徴量 X_train_sample (shape):", X_train_sample.shape)
print(X_train_sample)
print("\nサンプル目的変数 y_train_sample (shape):", y_train_sample.shape)
print(y_train_sample)

サンプル特徴量 X_train_sample (shape): (8, 2)
[[1 1]
 [1 2]
 [2 2]
 [2 3]
 [3 2]
 [3 4]
 [4 4]
 [4 5]]

サンプル目的変数 y_train_sample (shape): (8,)
[ 2.74112929  3.97267268  5.75458332  7.11499762  7.83681163  9.7050826
 12.1934408  12.74956042]


## 3. 重回帰モデルクラスの実装

`MultipleLinearRegression` クラスを作成します。

### 3.1 概念説明

*   **`__init__` メソッド**:
    *   回帰係数 `self.coefficients_` (β̂) と切片 `self.intercept_` (β̂₀) を格納するための変数を初期化します。

*   **`fit` メソッド**:
    *   **目的**: 訓練データ `X_train` と `y_train` を用いて、最適な回帰係数と切片を計算します。
    *   **処理の流れ**:
        1.  **計画行列の作成**: 入力された特徴量行列 `X_train` の先頭に、切片項に対応する「1」のみからなる列を追加します。これにより、計画行列 `X_b` (X with bias/intercept term) を作成します。
            `X_b = [1, X₁, X₂, ..., Xₚ]`
        2.  **正規方程式の計算**: `β̂ = (X_bᵀX_b)⁻¹X_bᵀY` の式に従って、回帰係数のベクトル `beta_hat` を計算します。
            *   `X_b.T`: `X_b` の転置
            *   `np.linalg.inv()`: 逆行列の計算
            *   `@` または `np.dot()`: 行列の積
        3.  **係数の保存**: 計算された `beta_hat` の最初の要素を切片 `self.intercept_` (β̂₀) とし、残りの要素を回帰係数 `self.coefficients_` (β̂₁, ..., β̂ₚ) として保存します。

*   **`predict` メソッド**:
    *   **目的**: 学習済みのモデル（切片と回帰係数）を使って、新しい特徴量データ `X_test` に対する目的変数の値を予測します。
    *   **処理の流れ**:
        1.  **計画行列の作成**: `fit` メソッドと同様に、`X_test` の先頭に切片項の「1」の列を追加して `X_b_test` を作成します。
        2.  **予測値の計算**: `Y_pred = X_b_test @ β̂` の式に従って予測値を計算します。ここで `β̂` は `fit` で学習した切片と回_係数を結合したベクトルです。
            具体的には、`y_pred = X_test @ self.coefficients_ + self.intercept_` としても計算できます。

### 3.2 `MultipleLinearRegression` クラスの実装

In [3]:
class MultipleLinearRegression:
    def __init__(self):
        self.intercept_ = None # 切片
        self.coef_ = None      # 回帰係数
        self.beta_hat = None   # 切片と回帰係数を含むベクトル

    def fit(self, X, y):
        '''
        訓練データを用いてモデルを学習する
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
            y(ndarray): 目的変数ベクトル (サンプル数,)
        '''

        # 計画行列の作成
        n_samples, n_features = X.shape
        X_b = np.c_[np.ones((n_samples, 1)), X]

        # 正規方程式を用いてβ_hatを計算
        try:
            # 標準的な逆行列
            beta_hat = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        except np.linalg.LinAlgError:
            # 逆行列が計算できない場合は擬似逆行列を使用
            beta_hat = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y

        # 計算された係数を保存
        self._beta_hat = beta_hat
        self.intercept_ = beta_hat[0]
        self.coef_ = beta_hat[1:]

    def predict(self, X):
        '''
        学習済みモデルを用いて予測を行う
        parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
        returns:
            ndarray: 予測値ベクトル (サンプル数,)
        '''

        if self.intercept_ is None or self.coef_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' before 'predict'.")
        
        # 計画行列の作成
        n_samples = X.shape[0]
        X_b_test = np.c_[np.ones((n_samples, 1)), X]

        # 予測値の計算
        y_pred = X_b_test @ self._beta_hat

        return y_pred

## 4. モデルの学習と予測 (サンプルデータ)

* **4.1 モデルのインスタンス化と学習**

    *   **目的**: `MultipleLinearRegression` クラスのインスタンスを作成し、サンプルデータ `X_train_sample`, `y_train_sample` を使ってモデルを学習させます。
    *   **処理**:
        1.  `model = MultipleLinearRegression()` でモデルオブジェクトを生成します。
        2.  `model.fit(X_train_sample, y_train_sample)` を呼び出し、モデルのパラメータ（切片と回帰係数）を計算させます。

* **4.2 訓練データに対する予測**    

    *   **目的**: 学習に使用したデータセット `X_train_sample` に対して予測を行い、モデルがどの程度データを再現できているかを確認します。
    *   **処理**:
        1.  `y_pred_sample = model.predict(X_train_sample)` を呼び出し、予測値を取得します。
        2.  予測値と実際の目的変数 `y_train_sample` を比較します。

* **4.3 新しいデータに対する予測**

    *   **目的**: モデルが学習していない新しいデータポイントに対して予測を行います。
    *   **処理**:
        1.  新しいデータポイント `X_new` を定義します。
        2.  `y_pred_new = model.predict(X_new)` を呼び出し、予測値を取得します。

In [4]:
# モデルのインスタンス化
mlr_model = MultipleLinearRegression()

# モデルの学習
mlr_model.fit(X_train_sample, y_train_sample)

print(f"学習済みモデルの切片 (β₀): {mlr_model.intercept_:.4f}")
print(f"学習済みモデルの回帰係数 (β₁, β₂): {mlr_model.coef_}")

学習済みモデルの切片 (β₀): -0.1698
学習済みモデルの回帰係数 (β₁, β₂): [2.00222924 1.01659612]


In [5]:
# 訓練データに対する予測
y_pred_train_sample = mlr_model.predict(X_train_sample)

print("\n訓練データに対する予測結果:")
for i in range(len(y_train_sample)):
    print(f"実測値: {y_train_sample[i]:.2f}, 予測値: {y_pred_train_sample[i]:.2f}, 誤差: {y_train_sample[i] - y_pred_train_sample[i]:.2f}")


訓練データに対する予測結果:
実測値: 2.74, 予測値: 2.85, 誤差: -0.11
実測値: 3.97, 予測値: 3.87, 誤差: 0.11
実測値: 5.75, 予測値: 5.87, 誤差: -0.11
実測値: 7.11, 予測値: 6.88, 誤差: 0.23
実測値: 7.84, 予測値: 7.87, 誤差: -0.03
実測値: 9.71, 予測値: 9.90, 誤差: -0.20
実測値: 12.19, 予測値: 11.91, 誤差: 0.29
実測値: 12.75, 予測値: 12.92, 誤差: -0.17


In [6]:
# 新しいデータポイントで予測
X_new_sample = np.array([
    [1, 3],  # X1=1, X2=3
    [3, 3],  # X1=3, X2=3
    [5, 1]   # X1=5, X2=1
])

y_pred_new_sample = mlr_model.predict(X_new_sample)

print("\n新しいデータに対する予測結果:")
for i in range(X_new_sample.shape[0]):
    print(f"入力 X = {X_new_sample[i]}, 予測 Y = {y_pred_new_sample[i]:.2f}")


新しいデータに対する予測結果:
入力 X = [1 3], 予測 Y = 4.88
入力 X = [3 3], 予測 Y = 8.89
入力 X = [5 1], 予測 Y = 10.86


## 5. より実践的なデータセットでの利用と評価

scikit-learnのデータセット（例: Boston Housing Prices や Diabetes Dataset）を使って、モデルの性能を評価することができます。
その際には、データの前処理（標準化など）、訓練データとテストデータへの分割、評価指標（MSE, R²など）の計算が重要になります。

ここでは、概念的な流れのみを示します。

### 5.1 データセットのロードと前処理

In [7]:
from sklearn.datasets import load_diabetes # 糖尿病データセットの読み込み
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# 特徴量の標準化 (重回帰では効果的な場合が多い)
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### 5.2 モデル学習、予測、評価 (概念)

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

model_practical = MultipleLinearRegression()
model_practical.fit(X_train, y_train) # 標準化されたデータで学習

y_pred_test = model_practical.predict(X_test)

# 評価
mse = mean_squared_error(y_test, y_pred_test) # または y_test_original_scale, y_pred_test_original_scale
r2 = r2_score(y_test, y_pred_test)         # または y_test_original_scale, y_pred_test_original_scale

print(f"\n実践的なデータセットでの評価:")
print(f"  切片: {model_practical.intercept_}")
print(f"  係数: {model_practical.coef_}")
print(f"  平均二乗誤差 (MSE): {mse:.4f}")
print(f"  決定係数 (R²): {r2:.4f}")


実践的なデータセットでの評価:
  切片: 151.34560453985992
  係数: [  1.80291044 -11.50907105  25.80070495  16.53858529 -44.30640613
  24.64170961   7.77309591  13.09553713  35.01740872   2.31502709]
  平均二乗誤差 (MSE): 2900.1936
  決定係数 (R²): 0.4526


## 6. 考察

*   このスクラッチ実装では、重回帰分析の基本的な計算である正規方程式 `β̂ = (XᵀX)⁻¹XᵀY` をNumPyを用いて直接計算しました。
*   **長所**:
    *   アルゴリズムの内部動作を理解しやすい。
    *   NumPyの行列演算により、比較的小規模なデータであれば効率的に計算可能。
*   **短所・注意点**:
    *   **多重共線性**: 説明変数間に強い相関がある（多重共線性）場合、`XᵀX` が正則でなくなり、逆行列 `(XᵀX)⁻¹` が計算できないか、非常に不安定になる可能性があります。この場合、疑似逆行列 (`np.linalg.pinv`) を使用したり、リッジ回帰やLASSO回帰のような正則化手法を検討する必要があります。（実装では、エラーハンドリングと疑似逆行列の使用の試みを含めました。）
    *   **計算コスト**: 特徴量数が非常に多い場合、`XᵀX` の逆行列計算は計算コストが高くなる可能性があります (`O(p³)`、pは特徴量数)。scikit-learnなどのライブラリは、より数値的に安定し、大規模データにも対応できるソルバー（例：勾配降下法、特異値分解(SVD)ベースの方法）を使用している場合があります。
    *   **特徴量のスケーリング**: 特徴量のスケールが大きく異なると、学習が不安定になったり、特定の係数が不当に大きくなったり小さくなったりすることがあります。事前に標準化などのスケーリングを行うことが推奨されます。